## Data Preprocessing

In [ ]:
#import packages
import pandas as pd

#Read datasets from CSSE github repo
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [2]:
#view the first 5 rows of each data frame
print(confirmed.head(5))
#print(deaths.head(5))
#print(recoveries.head(5))

  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  8/7/21  8/8/21  8/9/21  8/10/21  \
0        0        0        0        0  ...  149810  149810  150778   151013   
1        0        0        0        0  ...  133730  133912  133981   134201   
2        0        0        0        0  ...  180356  181376  182368   183347   
3        0        0        0        0  ...   14836   14836   14836    14873   
4        0        0        0        0  ...   43592   43662   43747    43890   

   8/11/21  8/12/21  8/13/21  8/14/21  8/15/21  8/16/21  
0   151291   151563   

Data sources from CSSE are in wide format. To better analyze data, we need to transform from wide to long.

In [3]:
# Transform wide format to long format
confirmed = pd.melt(confirmed, id_vars=confirmed.columns[:4], value_vars = confirmed.columns[4:], var_name = 'date', value_name = 'confirmed')
deaths = pd.melt(deaths, id_vars=deaths.columns[:4], value_vars = deaths.columns[4:], var_name = 'date', value_name = 'deaths')
recoveries = pd.melt(recoveries, id_vars=recoveries.columns[:4], value_vars = recoveries.columns[4:], var_name = 'date', value_name = 'recoveries')

In [4]:
#view the first 5 rows of each data frame
print(confirmed.head(5))
#print(deaths.head(5))
#print(recoveries.head(5))

  Province/State Country/Region       Lat       Long     date  confirmed
0            NaN    Afghanistan  33.93911  67.709953  1/22/20          0
1            NaN        Albania  41.15330  20.168300  1/22/20          0
2            NaN        Algeria  28.03390   1.659600  1/22/20          0
3            NaN        Andorra  42.50630   1.521800  1/22/20          0
4            NaN         Angola -11.20270  17.873900  1/22/20          0


The confirmed & deaths tables present Canada data by Province/State while the recoveries table only displays the total number of cases in the whole country. This conflict will need to addressed first before combining the tables to gether as unmatched join keys will be omitted.

In [5]:
#Create a filter to check just Canada for confirmed, deaths, and recoveries
filter1 = confirmed['Country/Region'].str.contains('Canada')
filter2 = deaths['Country/Region'].str.contains('Canada')
filter3 = recoveries['Country/Region'].str.contains('Canada')
stage1 = confirmed[filter1]
stage1.head()

,Province/State,Country/Region,Lat,Long,date,confirmed
39,Alberta,Canada,53.9333,-116.5765,1/22/20,0
40,British Columbia,Canada,53.7267,-127.6476,1/22/20,0
41,Diamond Princess,Canada,0.0000,0.0000,1/22/20,0
42,Grand Princess,Canada,0.0000,0.0000,1/22/20,0
43,Manitoba,Canada,53.7609,-98.8139,1/22/20,0


In [6]:
stage2 = deaths[filter2]
stage2.head()

,Province/State,Country/Region,Lat,Long,date,deaths
39,Alberta,Canada,53.9333,-116.5765,1/22/20,0
40,British Columbia,Canada,53.7267,-127.6476,1/22/20,0
41,Diamond Princess,Canada,0.0000,0.0000,1/22/20,0
42,Grand Princess,Canada,0.0000,0.0000,1/22/20,0
43,Manitoba,Canada,53.7609,-98.8139,1/22/20,0


In [7]:
stage3 = recoveries[filter3]
stage3.head()

,Province/State,Country/Region,Lat,Long,date,recoveries
39,NaN,Canada,56.1304,-106.3468,1/22/20,0
303,NaN,Canada,56.1304,-106.3468,1/23/20,0
567,NaN,Canada,56.1304,-106.3468,1/24/20,0
831,NaN,Canada,56.1304,-106.3468,1/25/20,0
1095,NaN,Canada,56.1304,-106.3468,1/26/20,0


In [8]:
#Summarize confirmed and deaths data by date
confirmed_canada = confirmed[confirmed['Country/Region'] == 'Canada'].groupby('date').sum()[['confirmed']]
deaths_canada = deaths[deaths['Country/Region'] == 'Canada'].groupby('date').sum()[['deaths']]
 
confirmed_canada.head()

,confirmed
date,
1/1/21,591149
1/10/21,666375
1/11/21,674624
1/12/21,681015
1/13/21,688097


In [9]:
#Extract columns from recoveries table
recoveries_canada = recoveries[recoveries['Country/Region'] == 'Canada']
canada_template = recoveries_canada[recoveries_canada.columns[:-1]].reset_index(drop=True)  
canada_template.head()

,Province/State,Country/Region,Lat,Long,date
0,NaN,Canada,56.1304,-106.3468,1/22/20
1,NaN,Canada,56.1304,-106.3468,1/23/20
2,NaN,Canada,56.1304,-106.3468,1/24/20
3,NaN,Canada,56.1304,-106.3468,1/25/20
4,NaN,Canada,56.1304,-106.3468,1/26/20


reset_index() is a method to reset the index of a data frame.

Syntax: DataFrame.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill="")

drop: Boolean value, adds the replaced index column to the data if False

## Combining Tables

In [10]:
#Join aggregated confirmed and deaths data with extracted columns
confirmed_canada = canada_template.merge(confirmed_canada, how='inner', left_on='date', right_index=True)
deaths_canada = canada_template.merge(deaths_canada, how='inner', left_on='date', right_index=True)
confirmed_canada.head()

,Province/State,Country/Region,Lat,Long,date,confirmed
0,NaN,Canada,56.1304,-106.3468,1/22/20,0
1,NaN,Canada,56.1304,-106.3468,1/23/20,0
2,NaN,Canada,56.1304,-106.3468,1/24/20,0
3,NaN,Canada,56.1304,-106.3468,1/25/20,0
4,NaN,Canada,56.1304,-106.3468,1/26/20,1


The left dataframe is canada_template, the right dataframe is confirmed_canada

left_on: label or list, or array-like 
* Column or index level names to join on in the left DataFrame.

right_index: bool, default False
* Use the index from the right DataFrame as the join key. Same caveats as left_index

In [11]:
#Add the aggregated data for Canada back to confirmed and deaths table
confirmed = confirmed[confirmed['Country/Region'] != 'Canada'].append(confirmed_canada)  #selected df without Canada
deaths = deaths[deaths['Country/Region'] != 'Canada'].append(deaths_canada)

confirmed.head()

,Province/State,Country/Region,Lat,Long,date,confirmed
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [12]:
#Check that the provinces are gone
filter1 = confirmed['Country/Region'].str.contains('Canada')
stage1 = confirmed[filter1]
stage1.head()

,Province/State,Country/Region,Lat,Long,date,confirmed
0,NaN,Canada,56.1304,-106.3468,1/22/20,0
1,NaN,Canada,56.1304,-106.3468,1/23/20,0
2,NaN,Canada,56.1304,-106.3468,1/24/20,0
3,NaN,Canada,56.1304,-106.3468,1/25/20,0
4,NaN,Canada,56.1304,-106.3468,1/26/20,1


In [13]:
#Join confirmed, deaths and recoveries data together
data = confirmed.merge(deaths, how='inner', on=['Country/Region', 'Province/State','date']).merge(recoveries,how = 'inner', on =['Country/Region', 'Province/State','date'])
data['recoveries'] = data['recoveries'].astype(int)
data = data[['Province/State', 'Country/Region', 'date', 'Lat', 'Long', 'confirmed', 'deaths', 'recoveries']]
data.sample(5)

,Province/State,Country/Region,date,Lat,Long,confirmed,deaths,recoveries
51127,Guadeloupe,France,8/3/20,16.265000,-61.551000,272,14,179
8435,NaN,Bahrain,2/23/20,26.027500,50.550000,0,0,0
25392,NaN,"Korea, South",4/27/20,35.907757,127.766922,10752,244,8854
112229,NaN,Palau,3/23/21,7.515000,134.582500,0,0,0
75882,NaN,Jamaica,11/5/20,18.109600,-77.297500,9373,217,4745


## Population Data
One metric used in the Covid-19 dashboard is infection rate: $confirmed / population$. Countries' population is not available in the CSSE dataset so we will need to combine with another source. 

Source: [Tanu N Prabhu](https://www.kaggle.com/tanuprabhu/population-by-country-2020)

One very common problems when combining different data sources is unmatched value names.

In [14]:
#Read dataset
population = pd.read_csv('https://raw.githubusercontent.com/cpepingco/COVID-19-Dash-/main/population_by_country_2020.csv')
population.sample(5)

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
168,Luxembourg,627704,1.66 %,10249,242,2590,9741.0,1.5,40,88 %,0.01 %
210,Saint Kitts & Nevis,53264,0.71 %,376,205,260,NaN,N.A.,N.A.,33 %,0.00 %
224,Tuvalu,11817,1.25 %,146,393,30,NaN,N.A.,N.A.,62 %,0.00 %
110,Kyrgyzstan,6542426,1.69 %,108345,34,191800,-4000.0,3.0,26,36 %,0.08 %
125,New Zealand,4829021,0.82 %,39170,18,263310,14881.0,1.9,38,87 %,0.06 %


In [15]:
for c in data['Country/Region'].unique():
    if c not in population['Country (or dependency)'].unique():
        print(c)

Burma
Congo (Brazzaville)
Congo (Kinshasa)
Cote d'Ivoire
Czechia
Diamond Princess
Korea, South
Kosovo
MS Zaandam
Saint Kitts and Nevis
Saint Vincent and the Grenadines
Sao Tome and Principe
Summer Olympics 2020
Taiwan*
US
West Bank and Gaza


A few of the countries have to be manually replaced.

In [16]:
country_mapper = {
    'Congo (Brazzaville)': 'Congo',
    'Congo (Kinshasa)': 'Congo',
    "Cote d'Ivoire": "Côte d'Ivoire",
    'Czechia': 'Czech Republic (Czechia)',
    'Korea, South': 'South Korea',
    'Saint Vincent and the Grenadines': 'St. Vincent & Grenadines',
    'Taiwan*': 'Taiwan',
    'US': 'United States',
    'West Bank and Gaza': 'Israel',
    'Saint Kitts and Nevis': 'Saint Kitts & Nevis',
    'Burma': 'Myanmar',
    'Sao Tome and Principe': 'Sao Tome & Principe'
}

data['Country/Region'] = data['Country/Region'].replace(country_mapper)
data.index = data['Country/Region']
data.sample(10)

,Province/State,Country/Region,date,Lat,Long,confirmed,deaths,recoveries
Country/Region,,,,,,,,
Sierra Leone,NaN,Sierra Leone,7/20/20,8.460555,-11.779889,1711,66,1261
Costa Rica,NaN,Costa Rica,4/23/20,9.748900,-83.753400,686,6,196
Australia,Victoria,Australia,8/3/20,-37.813600,144.963100,12335,147,5111
Côte d'Ivoire,NaN,Côte d'Ivoire,3/19/20,7.540000,-5.547100,9,0,1
Taiwan,NaN,Taiwan,5/27/21,23.700000,121.000000,6761,59,1133
Dominica,NaN,Dominica,12/7/20,15.415000,-61.371000,85,0,72
United Kingdom,Turks and Caicos Islands,United Kingdom,5/15/21,21.694000,-71.797900,2404,17,2365
China,Guangxi,China,5/25/21,23.829800,108.788100,275,2,268
Taiwan,NaN,Taiwan,5/2/20,23.700000,121.000000,432,6,324


In [17]:
#Export data
data.to_csv('covid19.csv')